# Integration Test Notebook

This notebook tests whether the standard project setup and configuration integration is working correctly across all notebooks in the nothreeinline-Spring-25 folder.

In [ ]:
# RLMath Framework Integration Setup
import sys
import os
import subprocess
from pathlib import Path

# Detect project root (should be the parent of nothreeinline-Spring-25)
current_dir = Path.cwd()
if current_dir.name == "nothreeinline-Spring-25":
    project_root = current_dir.parent
else:
    project_root = current_dir

print(f"📁 Project root detected: {project_root}")

# Add project root to Python path for imports
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"✅ Added {project_root} to Python path")

# Check Poetry environment
try:
    result = subprocess.run(['poetry', 'env', 'info', '--path'], 
                          capture_output=True, text=True, cwd=project_root)
    if result.returncode == 0:
        poetry_venv = result.stdout.strip()
        print(f"🎯 Poetry virtual environment: {poetry_venv}")
    else:
        print("⚠️ Not in a Poetry environment, using system Python")
except FileNotFoundError:
    print("⚠️ Poetry not found, using system Python")

# Standard imports
import numpy as np
import matplotlib.pyplot as plt
import toml
from itertools import combinations
import torch
import torch.nn as nn
from tqdm.notebook import trange, tqdm
import math
import random
import time
from collections import defaultdict

print("✅ Standard libraries imported successfully")

# Load project configuration using framework
config_path = project_root / "config.toml"
if config_path.exists():
    config = toml.load(config_path)
    print(f"✅ Loaded configuration from {config_path}")
    print(f"Environment: {config['env']['env_type']} ({config['env']['m']}x{config['env']['n']})")
    print(f"Algorithm: {config['algo']['method']}")
else:
    print("⚠️ No config.toml found, using default settings")
    config = {
        'env': {'m': 3, 'n': 10, 'env_type': 'NoThreeCollinearEnv'},
        'algo': {'method': 'dqn'},
        'train': {'episodes': 1000, 'batch_size': 32, 'lr': 0.0001}
    }

# Import RLMath framework modules
import_errors = []
framework_modules = {}

try:
    import src
    framework_modules['src'] = src
    print("✅ Main src module imported")
except ImportError as e:
    import_errors.append(f"src: {e}")

# Import registries (core framework components)
try:
    from src.registry import env_registry, algo_registry, model_registry
    framework_modules['registries'] = {
        'env_registry': env_registry,
        'algo_registry': algo_registry, 
        'model_registry': model_registry
    }
    print("✅ Framework registries imported")
except ImportError as e:
    import_errors.append(f"registries: {e}")

# Import environment classes via registry
try:
    from src.registry.env_registry import ENV_CLASSES
    framework_modules['env_classes'] = ENV_CLASSES
    print(f"✅ Environment classes: {list(ENV_CLASSES.keys())}")
except ImportError as e:
    import_errors.append(f"env_classes: {e}")

# Import algorithm classes via registry  
try:
    from src.registry.algo_registry import ALGO_CLASSES, EXTENDED_ALGO_CLASSES
    framework_modules['algo_classes'] = ALGO_CLASSES
    framework_modules['extended_algo_classes'] = EXTENDED_ALGO_CLASSES
    print(f"✅ Algorithm classes: {list(ALGO_CLASSES.keys())}")
except ImportError as e:
    import_errors.append(f"algo_classes: {e}")

# Import core framework modules
try:
    from src import envs, algos, models, utils
    framework_modules.update({
        'envs': envs,
        'algos': algos, 
        'models': models,
        'utils': utils
    })
    print("✅ Core framework modules imported")
except ImportError as e:
    import_errors.append(f"core_modules: {e}")

# Import advanced components with lazy loading
try:
    advanced_components = src.get_advanced_components()
    framework_modules['advanced'] = advanced_components
    print("✅ Advanced components loaded")
except (ImportError, AttributeError) as e:
    import_errors.append(f"advanced_components: {e}")

# Import specific modules commonly used in notebooks
try:
    from src.utils import count_idle_cpus, binomial, CPU_CORES
    framework_modules['cpu_utils'] = {'count_idle_cpus': count_idle_cpus, 'binomial': binomial, 'CPU_CORES': CPU_CORES}
    print("✅ CPU utilities imported")
except ImportError as e:
    import_errors.append(f"cpu_utils: {e}")

try:
    from src.priority_advanced import ensure_priority_grid_exists, load_priority_grid
    framework_modules['priority_advanced'] = {
        'ensure_priority_grid_exists': ensure_priority_grid_exists,
        'load_priority_grid': load_priority_grid
    }
    print("✅ Priority advanced functions imported")
except ImportError as e:
    import_errors.append(f"priority_advanced: {e}")

if import_errors:
    print("\n⚠️ Some import errors occurred:")
    for error in import_errors:
        print(f"  - {error}")
else:
    print("✅ All framework modules imported successfully")

# Framework-integrated utility functions
def create_environment(env_type=None, **kwargs):
    """Create an environment using the framework registry"""
    if env_type is None:
        env_type = config['env']['env_type']
    
    if 'env_classes' in framework_modules:
        env_class = framework_modules['env_classes'].get(env_type)
        if env_class:
            return env_class(**kwargs)
        else:
            raise ValueError(f"Environment type '{env_type}' not found in registry. Available: {list(framework_modules['env_classes'].keys())}")
    else:
        raise ImportError("Environment registry not available")

def create_algorithm(algo_type=None, **kwargs):
    """Create an algorithm using the framework registry"""
    if algo_type is None:
        algo_type = config['algo']['method']
    
    if 'algo_classes' in framework_modules:
        algo_class = framework_modules['algo_classes'].get(algo_type)
        if algo_class:
            return algo_class(**kwargs)
        else:
            raise ValueError(f"Algorithm type '{algo_type}' not found in registry. Available: {list(framework_modules['algo_classes'].keys())}")
    else:
        raise ImportError("Algorithm registry not available")

def are_collinear(p1, p2, p3):
    """Returns True if the three points are collinear."""
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3
    return (y1 - y2) * (x1 - x3) == (y1 - y3) * (x1 - x2)

def plot_points(points, n=None, title="No-3-in-line Set"):
    """Plot a set of points on a grid"""
    if not points:
        print("No points to plot.")
        return
    
    xs, ys = zip(*points)
    if n is None:
        n = max(max(xs), max(ys)) + 1
    
    plt.figure(figsize=(6, 6))
    plt.scatter(xs, ys, s=100, c='blue', edgecolors='black')
    plt.xticks(range(n))
    plt.yticks(range(n))
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.title(title)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim(-1, n)
    plt.ylim(-1, n)
    plt.show()

def get_framework_info():
    """Display current framework configuration and available components"""
    print("🔧 RLMath Framework Status:")
    print("=" * 50)
    print(f"Project Root: {project_root}")
    print(f"Config: {config}")
    print(f"Available Environments: {list(framework_modules.get('env_classes', {}).keys())}")
    print(f"Available Algorithms: {list(framework_modules.get('algo_classes', {}).keys())}")
    print(f"CPU Cores Available: {framework_modules.get('cpu_utils', {}).get('CPU_CORES', 'Unknown')}")
    print("=" * 50)

print("🚀 RLMath Framework integration complete!")
print(f"🔧 Working directory: {Path.cwd()}")
print(f"🏠 Project root: {project_root}")
print(f"🐍 Python executable: {sys.executable}")
print("\n💡 Use get_framework_info() to see available components")
print("💡 Use create_environment() and create_algorithm() to instantiate framework objects")

## Test 1: Configuration Loading

Let's verify that the configuration is loaded correctly and displays the expected values.

In [ ]:
# Test configuration loading
print("Configuration Test:")
print("=" * 50)
print(f"Project Root: {project_root}")
print(f"Environment Type: {config['env']['env_type']}")
print(f"Grid Size: {config['env']['m']} x {config['env']['n']}")
print(f"Algorithm: {config['algo']['method']}")
print(f"Training Episodes: {config['train']['episodes']}")
print(f"Batch Size: {config['train']['batch_size']}")
print(f"Learning Rate: {config['train']['lr']}")
print("=" * 50)

# Test framework integration
print("Framework Integration Test:")
print("=" * 50)

# Show framework info
get_framework_info()

# Test environment creation via registry
try:
    env = create_environment()
    print(f"✅ Successfully created environment: {type(env).__name__}")
    print(f"   Grid size: {env.m if hasattr(env, 'm') else 'Unknown'} x {env.n if hasattr(env, 'n') else 'Unknown'}")
except Exception as e:
    print(f"❌ Environment creation failed: {e}")

# Test algorithm creation via registry
try:
    algo = create_algorithm()
    print(f"✅ Successfully created algorithm: {type(algo).__name__}")
except Exception as e:
    print(f"❌ Algorithm creation failed: {e}")

print("=" * 50)

## Test 2: Module Imports

Test that all the key modules and classes are available.

In [ ]:
# Test module imports
print("Module Import Test:")
print("=" * 50)

# Test basic imports
try:
    assert np is not None
    print("✅ NumPy imported successfully")
except:
    print("❌ NumPy import failed")

try:
    assert plt is not None
    print("✅ Matplotlib imported successfully")
except:
    print("❌ Matplotlib import failed")

try:
    assert torch is not None
    print("✅ PyTorch imported successfully")
except:
    print("❌ PyTorch import failed")

# Test project modules
try:
    import src.envs.base_env
    print("✅ Base environment module imported")
except ImportError as e:
    print(f"❌ Base environment import failed: {e}")

try:
    import src.algos.mcts_advanced
    print("✅ Advanced MCTS module imported")
except ImportError as e:
    print(f"❌ Advanced MCTS import failed: {e}")

print("=" * 50)

## Test 3: Utility Functions

Test the common utility functions that are now standardized across notebooks.

In [ ]:
# Test utility functions
print("Utility Functions Test:")
print("=" * 50)

# Test are_collinear function
test_points = [
    [(0, 0), (1, 1), (2, 2)],  # Should be collinear
    [(0, 0), (1, 0), (0, 1)],  # Should not be collinear
    [(0, 0), (1, 2), (2, 4)]   # Should be collinear
]

for i, (p1, p2, p3) in enumerate(test_points):
    result = are_collinear(p1, p2, p3)
    expected = [True, False, True][i]
    status = "✅" if result == expected else "❌"
    print(f"{status} Test {i+1}: Points {p1}, {p2}, {p3} - Collinear: {result}")

print("=" * 50)

## Test 4: Environment Creation

Test creating an environment using the loaded configuration.

In [ ]:
# Test environment creation
print("Environment Creation Test:")
print("=" * 50)

try:
    # Try to create environment based on config
    env_type = config['env']['env_type']
    m, n = config['env']['m'], config['env']['n']
    
    print(f"Creating environment: {env_type} with grid size {m}x{n}")
    
    # Basic grid representation test
    grid = np.zeros((m, n))
    print(f"✅ Created {m}x{n} grid successfully")
    print(f"Grid shape: {grid.shape}")
    print(f"Total positions: {grid.size}")
    
    # Test some basic operations
    valid_moves = (grid.reshape(-1) == 0).astype(np.uint8)
    print(f"✅ Valid moves calculation works, found {np.sum(valid_moves)} valid positions")
    
except Exception as e:
    print(f"❌ Environment creation failed: {e}")

print("=" * 50)

## Test 5: Plotting Function

Test the standardized plotting function.

In [ ]:
# Test plotting function
print("Plotting Function Test:")
print("=" * 50)

# Create some test points
test_points = [(0, 0), (1, 2), (2, 1), (3, 3)]

try:
    print("Creating test plot...")
    plot_points(test_points, n=5, title="Integration Test - Sample Points")
    print("✅ Plotting function works correctly")
except Exception as e:
    print(f"❌ Plotting function failed: {e}")

print("=" * 50)

## Test 6: Notebook Integration Status

Check which notebooks have been successfully integrated with the standard setup.

In [ ]:
# Test notebook integration status
print("Notebook Integration Status:")
print("=" * 50)

notebook_dir = Path("/home/zxmath/RLMath/nothreeinline-Spring-25")
notebook_files = list(notebook_dir.glob("*.ipynb"))

integrated_count = 0
for notebook_path in notebook_files:
    try:
        with open(notebook_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        if "Standard Project Setup and Configuration" in content:
            integrated_count += 1
            status = "✅"
        else:
            status = "❌"
        
        print(f"{status} {notebook_path.name}")
    except Exception as e:
        print(f"❌ Error reading {notebook_path.name}: {e}")

print("=" * 50)
print(f"Integration Summary: {integrated_count}/{len(notebook_files)} notebooks integrated")

if integrated_count == len(notebook_files):
    print("🎉 ALL NOTEBOOKS SUCCESSFULLY INTEGRATED!")
else:
    print(f"⚠️ {len(notebook_files) - integrated_count} notebooks still need integration")

print("=" * 50)

## Test Summary

This notebook has tested the integration of standard project setup across all notebooks. The integration includes:

1. **Project path configuration** - Automatic detection and addition of project root to Python path
2. **Standard imports** - Common libraries (NumPy, PyTorch, Matplotlib, etc.)
3. **Configuration loading** - Automatic loading of `config.toml` with fallback defaults
4. **Module imports** - Import of all project modules (envs, algos, models, utils)
5. **Utility functions** - Common functions like `are_collinear` and `plot_points`

### Integration Benefits:
- ✅ **Consistency** - All notebooks now have the same setup
- ✅ **Maintainability** - Changes to imports/config only need to be made in one place
- ✅ **Reliability** - Proper error handling and fallbacks
- ✅ **Convenience** - Ready-to-use utility functions in every notebook

### Next Steps:
1. Run any notebook to verify it works with the new setup
2. Update individual notebooks to remove duplicate imports/setup code
3. Add any project-specific configurations as needed